In [37]:
import subprocess
import utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [38]:
mapDict={0:["corners",4],
         1:["corners2",5],
         2:["loop-chain",7],
         3:["connector",6],
         4:["string",5],
         5:["tree",3],
         6:["tree2",17],
         7:["tunnel",4],
         8:["lak303d",1000],
         9:["random-32-32-10-2",485],
         10:["random-32-32-10",460],
         11:["maze-128-128-1",350],
         12:["room-64-64-8",1000],
         13:["w_woundedcoast",1000],
         14:["Berlin_1_256",1000],
         15:["brc202d",1000],
         16:["den312d",400],
         17:["maze-32-32-4",100],
         18:["maze-128-128-2",300],
         19:["orz900d",1000],
         20:["warehouse-20-40-10-2-1",1000],
         21:["warehouse-20-40-10-2-2",1000],
         22:["room-32-32-4"],
         23:["warehouse-10-20-10-2-1"]}

Candidates=[8,10,11,12,13,14,15,16,17,18,19,20,21]

读取csv文件

In [46]:
import random
from tqdm import tqdm

df = pd.read_csv('result.csv')
# 提取特定列
columns = ['instance','total', 'n0', 'ub0', 'n4', 'ub4', 'average_ub0', 'average_ub4']
# 打印提取的 DataFrame
TieBreakers=['0','4']
selected=23
num=600
instance=mapDict[selected][0]+"&&"+str(num)
df=utils.check_and_add_instance(df,instance)
seeds_num=50
with tqdm(total=seeds_num, desc="Processing") as pbar:
    for i in range(seeds_num):
        seed=random.randint(0,100000000)
        # print(mapDict[selected][0])
        # print(num)
        # print(seed)
        df=utils.df_add_num(df,instance,'total',1)
        for tiebreaker in TieBreakers:
            # 构建命令
            compute = [
                '../build/main',
                '-i', f'../assets/{mapDict[selected][0]}.scen',
                '-m', f'../assets/{mapDict[selected][0]}.map',
                '-N', str(num),
                '-v', '1',
                '-t', '10',
                '--no-star',
                '--tie-breaker',tiebreaker,
                '-s',str(seed),
            ]
            try:
                # 运行命令
                result_compute = subprocess.run(compute, capture_output=True, text=True, check=True)
                # 打印输出
                # print("标准输出:")
                # print(result_compute.stdout)
                # print("标准错误:")
                # print(result_compute.stderr)
                cost,lb,ub=utils.extract_total_cost(result_compute.stdout)
                if ub>=1:
                    df=utils.df_add_num(df,instance,'n'+tiebreaker,1)
                    df=utils.df_add_num(df,instance,'ub'+tiebreaker,round(ub,5))
                    average_ub=df.loc[df['instance'] == instance, 'ub'+tiebreaker]/df.loc[df['instance'] == instance, 'n'+tiebreaker]
                    df=utils.df_set_num(df,instance,'average_ub'+tiebreaker,round(average_ub,5))
                    df.to_csv('result.csv', index=False)
                    pass
            except subprocess.CalledProcessError as e:
                print(f"命令执行失败: {e}")
                print(f"返回码: {e.returncode}")
                print(f"标准错误: {e.stderr}")
                break
        pbar.set_postfix(message=f"average_ub0:{df.loc[df['instance'] == instance, 'average_ub0'].values[0]},average_ub4:{df.loc[df['instance'] == instance, 'average_ub4'].values[0]}")
        #pbar.write(df.to_string())
        pbar.update(1)
print(df)


Processing: 100%|██████████| 50/50 [00:41<00:00,  1.21it/s, message=average_ub0:1.44507,average_ub4:1.39913]

                        instance  total   n0        ub0   n4        ub4  \
0           random-32-32-10&&400     51   51   96.49768   51   93.67884   
1            maze-128-128-1&&300     51   47  135.00067   50  142.42014   
2            maze-128-128-1&&200     50   50   96.75626   50   89.89157   
3            maze-128-128-1&&100     50   50   74.55139   50   73.05265   
4             Berlin_1_256&&1000     50   50   51.14929   50   51.07745   
5                   den312d&&400     50   50   91.72644   50   91.04574   
6                   den312d&&600     52   52  137.36559   52  131.94986   
7                   den312d&&800     57   57  217.33818   57  204.80594   
8                  den312d&&1000     50   50  241.59051   50  229.90559   
9   warehouse-20-40-10-2-1&&1000     50   50   55.55623   50   54.88429   
10                 brc202d&&2000     50   50   67.77552   50   66.35611   
11          random-32-32-10&&460    100  100  219.32012  100  210.87688   
12             maze-32-32